# Задача 1

Преобразуем базу данных sqlite в pandas dataframe и меняем тип объектов в столбцах 'data' и 'ts' на datetime

In [1]:
import sqlite3
con = sqlite3.connect("db.db")
cur = con.cursor()
import pandas as pd
pddb_spent = pd.read_sql("SELECT * FROM spent", con)
pddb_pay = pd.read_sql("SELECT * FROM payments", con)
pddb_spent.date = pddb_spent.date.apply(pd.to_datetime)
pddb_pay.date = pddb_pay.date.apply(pd.to_datetime)
pddb_pay.ts = pddb_pay.ts.apply(pd.to_datetime)

Находим макcимальное занчение date и ts

In [2]:
pddb_pay.index[pddb_pay['date'].argmax()]

1531

In [3]:
pddb_pay.loc[1531, ['date']]

date    2020-08-14 00:00:00
Name: 1531, dtype: object

In [4]:
pddb_pay.index[pddb_pay['ts'].argmax()]

22

In [5]:
pddb_pay.loc[22, ['ts']]

ts    2020-08-15 04:02:21.559178
Name: 22, dtype: object

Последняя дата, на которую есть информация по дате установки программы 14.08.2020, а последний день сбора данных из трекинговой системы 15.08.2020. Значит мы можем точно посчитать ARPPU и roas_28d для дат установки с 01.06.2020 по 18.07.2020 включительно (15.08.2020 - 28 дней). Для этого отберем из таблицы Payments информацию для соответствующих дат и посчитаем ARPPU как отношение суммы app_revenue к сумме unique_purchases по операционным системам. Получаем 2 числа, они записаны в файле res1.txt. Средний платящий пользователь ios платит заметно больше, чем таковой на android.

In [6]:
from datetime import timedelta, datetime
pddb_pay_28d = pddb_pay[(pddb_pay.ts.subtract(pddb_pay.date) >= timedelta(28))]
pddb_pay_28d = pddb_pay_28d.loc[pddb_pay_28d['app_revenue'] != 0]
pddb_pay_28d = pddb_pay_28d.sort_values(by=['date', 'os', 'ts', 'country'])
with open('pddb_pay_28d.txt', 'w') as f:
    f.write(pddb_pay_28d.to_string(header = True, index = False))
print('sum_revenue:', pddb_pay_28d.app_revenue.sum())

sum_revenue: 55802.103392000005


In [7]:
arppu_andr = pddb_pay_28d.loc[pddb_pay_28d['os'] != 'ios', 'app_revenue'].sum() / pddb_pay_28d.loc[pddb_pay_28d['os'] != 'ios', 'unique_purchases'].sum()
arppu_ios = pddb_pay_28d.loc[pddb_pay_28d['os'] == 'ios', 'app_revenue'].sum() / pddb_pay_28d.loc[pddb_pay_28d['os'] == 'ios', 'unique_purchases'].sum()
print('arppu_andr: ', arppu_andr)
print('arppu_ios: ', arppu_ios)
arppu = [arppu_andr, arppu_ios]
f = open('res1.txt', 'w')
f.write(str(arppu_andr)+" "+str(arppu_ios))
f.close()

arppu_andr:  17.841820437315636
arppu_ios:  28.121525693060498


Далее посчитаем суммы app_revenue по дням инсталла и операционным системам. В полученную таблицу добавим суммы средств потраченных на рекламу по дням расходования бюджета, чтобы посчитать roas_28d для каждой даты установки приложения с 01.06.2020 по 18.07.2020. Поскольку в данных трекинговой системы отсутствует информация в период с 27.06.2020 по 07.07.2020, попробуем их спрогнозировать основываясь на динамике показателя roas_28d.

In [8]:
roas_28d_count_andr = {}
roas_28d_count_ios = {}
dates_dict = {}
for i in pddb_pay_28d.date:
    count_andr = pddb_pay_28d.loc[(pddb_pay_28d['date'] == i) & (pddb_pay_28d['os'] != 'ios'), 'app_revenue'].sum()
    roas_28d_count_andr[i] = count_andr
    count_ios = pddb_pay_28d.loc[(pddb_pay_28d['date'] == i) & (pddb_pay_28d['os'] == 'ios'), 'app_revenue'].sum()
    roas_28d_count_ios[i] = count_ios
for i in pddb_pay_28d.date:
    dates_dict[i] = i

In [9]:
roas_28d = pd.DataFrame.from_dict(dates_dict, orient='index', columns=['date'])
roas_28d['sum_revenue_andr'] = pd.Series(roas_28d_count_andr)
roas_28d['sum_revenue_ios'] = pd.Series(roas_28d_count_ios)

In [11]:
roas_28d_count_andr = {}
roas_28d_count_ios = {}
count = 0
for i in pddb_pay_28d.date:
    count_andr = pddb_spent.loc[(pddb_spent['date'] == i) & (pddb_spent['os'] != 'ios'), 'spend'].sum()
    roas_28d_count_andr[i] = count_andr
    count_ios = pddb_spent.loc[(pddb_spent['date'] == i) & (pddb_spent['os'] == 'ios'), 'spend'].sum()
    roas_28d_count_ios[i] = count_ios

In [12]:
roas_28d['sum_spent_andr'] = pd.Series(roas_28d_count_andr)
roas_28d['sum_spent_ios'] = pd.Series(roas_28d_count_ios)

In [13]:
roas_28d['roas_28d_andr'] = roas_28d['sum_revenue_andr'] / roas_28d['sum_spent_andr']
roas_28d['roas_28d_ios'] = roas_28d['sum_revenue_ios'] / roas_28d['sum_spent_ios']

In [14]:
roas_28d = roas_28d.sort_values(by=['date'])

In [16]:
roas_28d

,date,sum_revenue_andr,sum_revenue_ios,sum_spent_andr,sum_spent_ios,roas_28d_andr,roas_28d_ios
2020-06-01,2020-06-01,2208.938254,1978.476830,2467.780046,2967.77000,0.895111,0.666654
2020-06-02,2020-06-02,3567.834776,3177.505735,3494.040010,3495.96000,1.021120,0.908908
2020-06-03,2020-06-03,2299.731097,1286.295997,3009.730000,2899.43000,0.764099,0.443638
2020-06-04,2020-06-04,1693.687620,1772.849179,1905.960000,2415.45000,0.888627,0.733962
2020-06-05,2020-06-05,1497.188086,1668.396440,1504.110000,1860.82000,0.995398,0.896592
2020-06-06,2020-06-06,1989.892110,2274.436798,2152.330000,2635.68000,0.924529,0.862941
2020-06-07,2020-06-07,2034.304230,2986.215840,2234.040000,2810.11977,0.910594,1.062665
2020-06-08,2020-06-08,1199.214612,2323.964857,1549.220000,2065.36000,0.774076,1.125211
2020-06-09,2020-06-09,672.994146,1807.677200,1057.439984,1444.27000,0.636437,1.251620
2020-06-10,2020-06-10,736.576090,1723.352730,1059.900000,1525.44000,0.694949,1.129741


In [17]:
roas_28d_june_andr = roas_28d.loc[(roas_28d['date'] >= datetime(2020,6,1)) & (roas_28d['date'] < datetime(2020,7,1)), 'sum_revenue_andr'].sum() / roas_28d.loc[(roas_28d['date'] >= datetime(2020,6,1)) & (roas_28d['date'] < datetime(2020,7,1)), 'sum_spent_andr'].sum()
print('roas_28d_june_andr', roas_28d_june_andr)
roas_28d_june_ios = roas_28d.loc[(roas_28d['date'] >= datetime(2020,6,1)) & (roas_28d['date'] < datetime(2020,7,1)), 'sum_revenue_ios'].sum() / roas_28d.loc[(roas_28d['date'] >= datetime(2020,6,1)) & (roas_28d['date'] < datetime(2020,7,1)), 'sum_spent_ios'].sum()
print('roas_28d_june_ios', roas_28d_june_ios)

roas_28d_july_andr = roas_28d.loc[(roas_28d['date'] >= datetime(2020,7,1)) & (roas_28d['date'] < datetime(2020,7,19)), 'sum_revenue_andr'].sum() / roas_28d.loc[(roas_28d['date'] >= datetime(2020,7,1)) & (roas_28d['date'] < datetime(2020,7,19)), 'sum_spent_andr'].sum()
print('roas_28d_july_andr', roas_28d_july_andr)
roas_28d_july_ios = roas_28d.loc[(roas_28d['date'] >= datetime(2020,7,1)) & (roas_28d['date'] < datetime(2020,7,19)), 'sum_revenue_ios'].sum() / roas_28d.loc[(roas_28d['date'] >= datetime(2020,7,1)) & (roas_28d['date'] < datetime(2020,7,19)), 'sum_spent_ios'].sum()
print('roas_28d_july_ios', roas_28d_july_ios)

K_grouth_andr = (roas_28d_july_andr - roas_28d_june_andr) / roas_28d_june_andr
K_grouth_ios = (roas_28d_july_ios - roas_28d_june_ios) / roas_28d_june_ios
print('K_grouth_andr', K_grouth_andr)
print('K_grouth_ios', K_grouth_ios)
f = open('res2.txt', 'w')
f.write(str(K_grouth_andr)+" "+str(K_grouth_ios))
f.close()

roas_28d_june_andr 0.88568661400334
roas_28d_june_ios 0.8413998010949616
roas_28d_july_andr 0.6533901578727626
roas_28d_july_ios 1.163261489762549
K_grouth_andr -0.26227838657353975
K_grouth_ios 0.3825312155395455


Коэффициенты роста (снижения при отрицательном значении) roas_28d для android и ios операционных систем установки приложения в июле по сравнению с июнем сохранены в прилагающийся файл res2.txt.
Сначала хотел построить график зависимости roas_28d_andr и roas_28d_ios от даты инсталла, чтобы прикинуть недостающие значения в начале и в конце июля... Но, мне кажется, это не имеет смысла... В целом, очевидно, что roas_28d меньше 1.2 почти по всем дням инсталла,и в среднем по месяцам, и показатели июля хуже, чем июня. Однако roas_28d для ios значительно вырос в июле по сравнению с июнем, вероятно, это вызвано увеличением числа конверсий установок приложения в покупки, что будет показано ниже. Но при существенном снижении абсолютных показателей выручки в июле по сравнению с июнем, не вижу повода сильно радоваться увеличению относительного показателя. Можно попробовать проанализировать, что особенного происходило в даты установки, по которым показатели roas_28d высокие, возможно, есть отличия в видах рекламы, или еще что-то, что сработало, и надо это внедрять шире. Считаю необходимым в первую очередь разбираться с тем, в каких странах и на каких операционных системах рекламная компания окупается и приносит результаты. Для этого составим таблицу, в которой для каждой страны найдем суммарные значения app_revenue и spent и roas_28d от них.

In [18]:
countries_all = pddb_spent['country'].to_list()
countries_all = list(set(countries_all))
countries_all.sort()

In [19]:
pddb_spent_28d = pddb_spent.loc[pddb_spent['date'] < datetime(2020,7,19)]
with open('pddb_spent_28d.txt', 'w') as f:
    f.write(pddb_spent_28d.to_string(header = True, index = False))

In [20]:
pddb_joined = pd.merge(pddb_pay_28d, pddb_spent_28d, how='outer', on=['date', 'country', 'os'])
pddb_joined = pddb_joined.fillna(0)
pddb_joined = pddb_joined.loc[pddb_joined['impressions'] != 0]
pddb_joined = pddb_joined.sort_values(by=['country', 'os','date'])
with open('pddb_joined.txt', 'w') as f:
    f.write(pddb_joined.to_string(header = True, index = False))


In [21]:
pddb_joined_andr = pddb_joined.loc[pddb_joined['os'] != 'ios']
columns = ['purchases', 'unique_purchases', 'installs', 'app_revenue', 'spend', 'impressions', 'clicks']

countries_summary_andr = pd.DataFrame(countries_all, columns =['country'])

for clmn in columns:
    count = 0
    count_list = []
    for cntry in countries_all:
        count = pddb_joined_andr.loc[pddb_joined_andr['country'] == cntry, clmn].sum()
        count_list.append(count)
    countries_summary_andr[clmn] = pd.Series(count_list)

countries_summary_andr['roas28d'] = countries_summary_andr['app_revenue'] / countries_summary_andr['spend']
countries_summary_andr['profit'] = countries_summary_andr['app_revenue'] - countries_summary_andr['spend']

countries_summary_andr = countries_summary_andr.sort_values(by=['roas28d'], ascending=False)
    
with open('countries_summary_andr.txt', 'w') as f:
    f.write(countries_summary_andr.to_string(header = True, index = False))

In [22]:
pddb_joined_ios = pddb_joined.loc[pddb_joined['os'] == 'ios']
columns = ['purchases', 'unique_purchases', 'installs', 'app_revenue', 'spend', 'impressions', 'clicks']

countries_summary_ios = pd.DataFrame(countries_all, columns =['country'])

for clmn in columns:
    count = 0
    count_list = []
    for cntry in countries_all:
        count = pddb_joined_ios.loc[pddb_joined_ios['country'] == cntry, clmn].sum()
        count_list.append(count)
    countries_summary_ios[clmn] = pd.Series(count_list)
    
countries_summary_ios['roas28d'] = countries_summary_ios['app_revenue'] / countries_summary_ios['spend']
countries_summary_ios['profit'] = countries_summary_ios['app_revenue'] - countries_summary_ios['spend']

countries_summary_ios = countries_summary_ios.sort_values(by=['roas28d'], ascending=False)
    
with open('countries_summary_ios.txt', 'w') as f:
    f.write(countries_summary_ios.to_string(header = True, index = False))

In [23]:
pddb_joined_andr.to_excel('./pddb_joined_andr.xlsx')

In [24]:
pddb_joined_ios.to_excel('./pddb_joined_ios.xlsx')

Проанализируем полученные таблицы countries_summary_andr и countries_summary_ios (файлы с соответствующими именами .txt прилагаются). Они отсортированы по roas_28d. Очевидно, что странами, где проведены самые эффективные и прибыльные рекламные кампании являются CH, AU и RU для приложения на android, и BE, DE, FR для приложения на ios. Roas_28d больше 1,2 для 21 страны для приложения на android и для 19 стран для приложения на ios. Но в некоторых из них абсолютные цифры выручки и прибыли очень малы, поэтому не стоит на них обращать большого внимания и делать ставку, просто поддерживать имеющийся уровень и, может быть, немного его повышать. Считаю необходимым уделить основное внимание странам типа US где большие объемы продаж, но нет прибыли из-за ещё бОльших вложений в рекламу. Можно посмотреть данную таблицу в динамике, например, в разбивке по неделям (поскольку по месяцам нет полных данных за июль, мне кажется, по неделям будет более информативно) и понять тренд - уменьшаются ли расходы на рекламу и что при этом происходит с выручкой от продаж. По-моему, этот анализ выходит за рамки Задач, если нужно, его можно быстро провести на основе имеющегося кода.


Построим воронку конверсий.

In [25]:
def sum (df, os, date_start, date_end, column):
    sum = df.loc[(df['os'] == os) & (df.date >= date_start) & (df.date < date_end), column].sum()
    return sum

sum_impressions_june_andr = sum(pddb_spent, 'android', datetime(2020,6,1), datetime(2020,7,1), 'impressions')
print('sum_impressions_june_andr: ', sum_impressions_june_andr)
sum_impressions_june_ios = sum(pddb_spent, 'ios', datetime(2020,6,1), datetime(2020,7,1), 'impressions')
print('sum_impressions_june_ios: ', sum_impressions_june_ios)

sum_clicks_june_andr = sum(pddb_spent, 'android', datetime(2020,6,1), datetime(2020,7,1), 'clicks')
print('sum_clicks_june_andr: ', sum_clicks_june_andr)
sum_clicks_june_ios = sum(pddb_spent, 'ios', datetime(2020,6,1), datetime(2020,7,1), 'clicks')
print('sum_clicks_june_ios: ', sum_clicks_june_ios)

sum_installs_june_andr = sum(pddb_spent, 'android', datetime(2020,6,1), datetime(2020,7,1), 'installs')
print('sum_installs_june_andr: ', sum_installs_june_andr)
sum_installs_june_ios = sum(pddb_spent, 'ios', datetime(2020,6,1), datetime(2020,7,1), 'installs')
print('sum_installs_june_ios: ', sum_installs_june_ios)

pddb_pay_28d = pddb_pay[(pddb_pay.ts.subtract(pddb_pay.date) >= timedelta(28))]

sum_purchases_june_andr = sum(pddb_pay_28d, 'android', datetime(2020,6,1), datetime(2020,7,1), 'purchases')
print('sum_purchases_june_andr: ', sum_purchases_june_andr)
sum_purchases_june_ios = sum(pddb_pay_28d, 'ios', datetime(2020,6,1), datetime(2020,7,1), 'purchases')
print('sum_purchases_june_ios: ', sum_purchases_june_ios)

sum_impressions_july_andr = sum(pddb_spent, 'android', datetime(2020,7,1), datetime(2020,7,19), 'impressions')
print('sum_impressions_july_andr: ', sum_impressions_july_andr)
sum_impressions_july_ios = sum(pddb_spent, 'ios', datetime(2020,7,1), datetime(2020,7,19), 'impressions')
print('sum_impressions_july_ios: ', sum_impressions_july_ios)

sum_clicks_july_andr = sum(pddb_spent, 'android', datetime(2020,7,1), datetime(2020,7,19), 'clicks')
print('sum_clicks_july_andr: ', sum_clicks_july_andr)
sum_clicks_july_ios = sum(pddb_spent, 'ios', datetime(2020,7,1), datetime(2020,7,19), 'clicks')
print('sum_clicks_july_ios: ', sum_clicks_july_ios)

sum_installs_july_andr = sum(pddb_spent, 'android', datetime(2020,7,1), datetime(2020,7,19), 'installs')
print('sum_installs_july_andr: ', sum_installs_july_andr)
sum_installs_july_ios = sum(pddb_spent, 'ios', datetime(2020,7,1), datetime(2020,7,19), 'installs')
print('sum_installs_july_ios: ', sum_installs_july_ios)

sum_purchases_july_andr = sum(pddb_pay_28d, 'android', datetime(2020,7,1), datetime(2020,7,19), 'purchases')
print('sum_purchases_july_andr: ', sum_purchases_july_andr)
sum_purchases_july_ios = sum(pddb_pay_28d, 'ios', datetime(2020,7,1), datetime(2020,7,19), 'purchases')
print('sum_purchases_july_ios: ', sum_purchases_july_ios)

sum_impressions_june_andr:  7726096
sum_impressions_june_ios:  5794299
sum_clicks_june_andr:  145263
sum_clicks_june_ios:  88506
sum_installs_june_andr:  53271
sum_installs_june_ios:  25055
sum_purchases_june_andr:  7109
sum_purchases_june_ios:  5978
sum_impressions_july_andr:  522560
sum_impressions_july_ios:  660748
sum_clicks_july_andr:  9204
sum_clicks_july_ios:  10084
sum_installs_july_andr:  3633
sum_installs_july_ios:  2917
sum_purchases_july_andr:  750
sum_purchases_july_ios:  1625


In [26]:
conv_1_june_andr = sum_clicks_june_andr / sum_impressions_june_andr *100
print('conv_1_june_andr: ', conv_1_june_andr)

conv_1_june_ios = sum_clicks_june_ios / sum_impressions_june_ios *100
print('conv_1_june_ios: ', conv_1_june_ios)

conv_2_june_andr = sum_installs_june_andr / sum_clicks_june_andr *100
print('conv_2_june_andr: ', conv_2_june_andr)

conv_2_june_ios = sum_installs_june_ios / sum_clicks_june_ios *100
print('conv_2_june_ios: ', conv_2_june_ios)

conv_3_june_andr = sum_purchases_june_andr / sum_installs_june_andr *100
print('conv_3_june_andr: ', conv_3_june_andr)

conv_3_june_ios = sum_purchases_june_ios / sum_installs_june_ios *100
print('conv_3_june_ios: ', conv_3_june_ios)

conv_1_june_andr:  1.8801604329016879
conv_1_june_ios:  1.527466911873205
conv_2_june_andr:  36.67210507837508
conv_2_june_ios:  28.308815221566903
conv_3_june_andr:  13.344971935950142
conv_3_june_ios:  23.859509080023948


In [27]:
conv_1_july_andr = sum_clicks_july_andr / sum_impressions_july_andr *100
print('conv_1_july_andr: ', conv_1_july_andr)

conv_1_july_ios = sum_clicks_july_ios / sum_impressions_july_ios *100
print('conv_1_july_ios: ', conv_1_july_ios)

conv_2_july_andr = sum_installs_july_andr / sum_clicks_july_andr *100
print('conv_2_july_andr: ', conv_2_july_andr)

conv_2_july_ios = sum_installs_july_ios / sum_clicks_july_ios *100
print('conv_2_july_ios: ', conv_2_july_ios)

conv_3_july_andr = sum_purchases_july_andr / sum_installs_july_andr *100
print('conv_3_july_andr: ', conv_3_july_andr)

conv_3_july_ios = sum_purchases_july_ios / sum_installs_july_ios *100
print('conv_3_july_ios: ', conv_3_july_ios)

conv_1_july_andr:  1.761328842620943
conv_1_july_ios:  1.5261491521729917
conv_2_july_andr:  39.47196870925684
conv_2_july_ios:  28.927013090043634
conv_3_july_andr:  20.644095788604456
conv_3_july_ios:  55.70791909496058


In [28]:
delta_impressions_andr = (sum_impressions_july_andr - sum_impressions_june_andr) / sum_impressions_june_andr * 100
print('delta_impressions_andr', delta_impressions_andr)
delta_impressions_ios = (sum_impressions_july_ios - sum_impressions_june_ios) / sum_impressions_june_ios * 100
print('delta_impressions_ios', delta_impressions_ios)

delta_conv_1_andr = (conv_1_july_andr - conv_1_june_andr) / conv_1_june_andr * 100
delta_conv_1_ios = (conv_1_july_ios - conv_1_june_ios) / conv_1_june_ios * 100
print('delta_conv_1_andr: ', delta_conv_1_andr)
print('delta_conv_1_ios: ', delta_conv_1_ios)

delta_conv_2_andr = (conv_2_july_andr - conv_2_june_andr) / conv_2_june_andr * 100
delta_conv_2_ios = (conv_2_july_ios - conv_2_june_ios) / conv_2_june_ios * 100
print('delta_conv_2_andr: ', delta_conv_2_andr)
print('delta_conv_2_ios: ', delta_conv_2_ios)

delta_conv_3_andr = (conv_3_july_andr - conv_3_june_andr) / conv_3_june_andr * 100
delta_conv_3_ios = (conv_3_july_ios - conv_3_june_ios) / conv_3_june_ios * 100
print('delta_conv_3_andr: ', delta_conv_3_andr)
print('delta_conv_3_ios: ', delta_conv_3_ios)

delta_impressions_andr -93.23642885100055
delta_impressions_ios -88.59658433228937
delta_conv_1_andr:  -6.320289917884814
delta_conv_1_ios:  -0.08627091624506378
delta_conv_2_andr:  7.634859315814929
delta_conv_2_ios:  2.183764539908263
delta_conv_3_andr:  54.695685293958086
delta_conv_3_ios:  133.48309015126085


Переменные delta_ это изменения показателей в июле по сравнению с июнем в процентах. Видно, что в июле резко упало число просмотров по сравнению с июнем. Это нельзя объяснить отсутствием части даных в июле, так как данных нет чуть более, чем по половине месяца, а число просмотров упало почти в 15 раз по android и почти в 9 раз по ios. Нужно понять, чем это вызвано... Рекламный бюджет на столько не сократился. Возможно, стали использоваться не эффективные каналы рекламы или есть какие-то другие причины, необходимо разбираться. Конверсия из просмотров в клики незначительно уменьшилась, из кликов в инсталлы - незначительно увеличилась. А вот конверсия из инсталлов в покупки существенно возрасла, особенно для ios, тут компания движется  вправильном направлении.

     Какие можно дать рекомендации отделу по закупке трафика?... На основе имеющихся у меня данных, можно порекомендовать следующее:
    1. Закупать рекламный трафик в странах, в которых хорошие показатели roas и выручки одновременно, для соответствующей операционной системы.
    2. Не наращивать или немного наращивать рекламный трафик в странах с хорошим roas, но низкой выручкой, вероятно в них просто маленький рынок и нет смысла много вкладывать в рекламу.
    3. Вкладывать по-немногу и оперативно отслеживать отдачу в странах с roas больше примерно 0,8 и меньше 1,2
    4. Отказаться от вложений в рекламный трафик в странах, где roas меньше примерно 0,8
    5. Особый случай - страны типа US, обладающие большим рынком сбыта, но не приносящие прибыль на данный момент. Надо постараться понять, как выйти на прибыль в них, глубоко проанализировать все нюансы именно по ним, т.к. тут можно хорошо заработать, если научиться доходить до покупателя с минимальными затратами на рекламу.
     Ещё один момент - очень низкая конверсия из просмотров в клики, всего 1,5-1,9 процента... Надо разобраться, почему так, возможно, деньги вкладываются в рекламу не для целевой аудитории... Или рекламные материалы недостаточно привлекательны для потенциальных клиентов.
    Обладая более полной информацией и видя картину шире изнутри, конечно, можно дать более точные рекомендации.
    В реальных условия, конечно же, нужно учитывать расходы фирмы - может оказаться, что даже показателя roas_28d = 1.2 будет недостаточно для получения прибыли организацией из-за высоких расходов, например, на аренду, оплату труда, налоги и т.д.
    